<a href="https://colab.research.google.com/github/saiku122/AIJobcolle/blob/master/MachineLearning/finaltask/LatstTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/saiku122/AIJobcolle.git

Cloning into 'AIJobcolle'...
remote: Enumerating objects: 339, done.
remote: Counting objects: 100% (339/339), done.
remote: Compressing objects: 100% (224/224), done.
remote: Total 339 (delta 136), reused 191 (delta 62), pack-reused 0
Receiving objects: 100% (339/339), 11.86 MiB | 16.02 MiB/s, done.
Resolving deltas: 100% (136/136), done.


★★最終課題★★</br>
〇入力データ</br>
・モデル用：final_hr_analysis_train.csv</br>
・スコア用：final_hr_analysis_test.csv</br>
</br>
〇出力データ</br>
・スコア付データ：</br> 
・モデル：</br>


In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import RFECV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
import joblib

In [4]:
file_model_name = 'final_hr_analysis_train.csv'
file_test_name = 'final_hr_analysis_test.csv'

#データの読み込み
df = pd.read_csv('./input/'+ file_model_name, header=0)

FileNotFoundError: ignored